# **Projet Statistique Bayésienne**
## *Bayesian Variable Selection and Estimation for Group Lasso*
#### Import Libraries

In [1]:
import random
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import exp, sqrt, log
from scipy import stats
from sklearn.linear_model import LinearRegression
import asgl

random.seed()
np.random.seed(0)

In [2]:
# Remove useless warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import warnings
warnings.filterwarnings('ignore')

#### Simulate DataSets

In [3]:
class SimulateDataSet():
    def __init__(self, n_obs, n_cov, n_groups, n_train, beta, corr, sigma, example):
        self.n_obs = n_obs
        self.n_cov = n_cov
        self.n_groups = n_groups
        self.n_train = n_train
        self.beta = beta
        self.corr = corr
        self.sigma = sigma
        self.example = example
    
    def create_corr_matrix(self):
        corr1 = pd.DataFrame(data=self.rand1).corr()
        mat_0_5 = np.full_like(corr1, self.corr)
        mat_diag = np.eye(corr1.shape[0], corr1.shape[1]) * (1 - self.corr)
        corr_target = mat_diag + mat_0_5
        self.corr_target_list = [list(xi) for xi in corr_target]
    
    def create_X(self):
        if self.example == 1:
            self.rand1 = np.random.rand(self.n_obs, self.n_cov)
            self.create_corr_matrix()

            L = np.linalg.cholesky(self.corr_target_list)
            correlated = np.dot(L, self.rand1.T)
            self.X = correlated.T

        elif self.example == 2:
            self.X = np.empty((self.n_obs, self.n_cov))
            group_size = self.n_cov // self.n_groups
            for g in range(self.n_groups):
                z_g = np.random.normal(size=(self.n_obs, 1))
                Z = np.concatenate([z_g] * group_size, axis=1)
                z_gj = np.random.normal(size=(self.n_obs, 1))
                for _ in range (1, group_size):
                    z_gj = np.concatenate([z_gj, np.random.normal(size=(self.n_obs, 1))], axis=1)
                predictors = Z + z_gj
                self.X[:, g * group_size : (g + 1) * group_size] = predictors
        return self.X

    
    def simulate_Y(self):
        self.residu = np.random.default_rng().normal(0, self.sigma, self.n_obs).reshape(self.n_obs, 1)
        self.Y = self.X.dot(self.beta.T) + self.residu
        return self.Y
    
    def create_train_test_set(self):
        test_size = 1 - self.n_train / self.n_obs
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.Y, 
                                                                                test_size=test_size, 
                                                                                random_state=42)
        return self.X_train, self.X_test, self.y_train, self.y_test


In [4]:
beta1 = np.array([[0.3, -1, 0, 0.5, 0.01] + [0]*5 + [0.8]*5 + [0]*5])

example1 = SimulateDataSet(n_obs=100, n_cov=20, n_groups=4, n_train=60, 
                           beta=beta1, corr=0.5, sigma=3, example=1)
X1 = example1.create_X()
Y1 = example1.simulate_Y()
n_groups1 = example1.n_groups
X_train1, X_test1, y_train1, y_test1 = example1.create_train_test_set()

group_size1 = [5] * 4

In [5]:
beta2 = np.array([[1, 2, 3, 4, 5] + [0]*5 + [0.1, 0.2, 0.3, 0.4, 0.5] + 13*([0]*5)])

example2 = SimulateDataSet(n_obs=60, n_cov=80, n_groups=16, n_train=40, 
                           beta=beta2, corr=None, sigma=2, example=2)
X2 = example2.create_X()
Y2 = example2.simulate_Y()
n_groups2 = example2.n_groups
X_train2, X_test2, y_train2, y_test2 = example2.create_train_test_set()

group_size2 = [5] * 16

In [6]:
beta3 = np.array([[0]*10 + [2]*10 + [0]*10 + [2]*10])

example3 = SimulateDataSet(n_obs=100, n_cov=40, n_groups=4, n_train=60, 
                           beta=beta3, corr=None, sigma=2, example=2)
X3 = example3.create_X()
Y3 = example3.simulate_Y()
n_groups3 = example3.n_groups
X_train3, X_test3, y_train3, y_test3 = example3.create_train_test_set()

group_size3 = [10] * 4

In [7]:
beta4 = np.array([[0]*10 + [2]*5 + [0]*5 + [0]*10 + [2]*5 + [0]*5])

example4 = SimulateDataSet(n_obs=100, n_cov=40, n_groups=4, n_train=60, 
                           beta=beta4, corr=None, sigma=2, example=2)
X4 = example4.create_X()
Y4 = example4.simulate_Y()
n_groups4 = example4.n_groups
X_train4, X_test4, y_train4, y_test4 = example4.create_train_test_set()

group_size4 = [10] * 4

#### Compute BGL-SS

In [8]:
class BGLSS():
    def __init__(self, Y, X, group_size, niter=10000, burnin=5000, a=1, b=1, num_update=100,
                 niter_update=100, verbose=False, alpha=1e-1, gamma=1e-1, pi=0.5):
        self.Y = Y
        self.X = X
        self.group_size = group_size
        self.niter = niter
        self.burnin = burnin
        self.a = a
        self.b = b
        self.num_update = num_update
        self.niter_update = niter_update
        self.verbose = verbose
        self.alpha = alpha
        self.gamma = gamma
        self.pi = pi

        # initialize values
        self.n = len(self.Y)
        self.p = self.X.shape[1]
        self.ngroup = len(self.group_size)
    

    def _initialize_gibbs_sampler(self):
        self.tau2 = np.ones(self.ngroup)
        self.sigma2 = 1
        self.l = np.zeros(self.ngroup)
        self.beta = [np.zeros(g) for g in self.group_size]
        self.Z = np.zeros(self.ngroup)


    def _initialize_EM_lambda(self):
        self.tau2 = np.ones(self.ngroup)
        self.sigma2 = 1
        self.lambda2 = 1
        self.matlambda2 = np.ones(self.ngroup)
        self.lambda2_path = np.ones(self.num_update) * (-1)
        self.matlambda2_path = np.ones((self.ngroup, self.num_update) * (-1))
        self.l = np.zeros(self.ngroup)
        self.beta = [np.zeros(g) for g in self.group_size]
        self.Z = np.zeros(self.ngroup)


    def _get_matrix_computation(self):
        self.X_not_g = [np.concatenate([self.X[:, : g * self.group_size[g]], 
                                        self.X[:, (g+1) * self.group_size[g]:]], axis=1) 
                                        for g in range(self.ngroup)]
        self.X_g = [self.X[:, g * self.group_size[g]:(g+1) * self.group_size[g]] 
                    for g in range(self.ngroup)]

        self.YtY = self.Y.T.dot(self.Y)
        self.XtY = self.X.T.dot(self.Y)
        self.XtX = self.X.T.dot(self.X)

        self.XgtY = [self.X_g[g].T.dot(self.Y) for g in range(self.ngroup)]
        self.XgtXg = [self.X_g[g].T.dot(self.X_g[g]) for g in range(self.ngroup)]
        self.XgtXng = [self.X_g[g].T.dot(self.X_not_g[g]) for g in range(self.ngroup)]
    

    def _update_beta(self, i):
        self.bng = np.concatenate([self.beta[j] for j in range(self.ngroup) if j != i])
        self.bng = self.bng.reshape(len(self.bng), 1)

        self.f1 = self.XgtY[i] - self.XgtXng[i].dot(self.bng)
        self.f2 = self.XgtXg[i] + np.eye(self.group_size[i])/self.tau2[i]
        self.f2_inverse = np.linalg.inv(self.f2)
        self.mu = (self.f2_inverse.dot(self.f1)).T[0]
                
        maxf = np.max(self.f2)
        trythis = (-self.group_size[i]/2)*np.log(self.tau2[i])\
                   + (-1/2)*np.log(np.linalg.det(self.f2/maxf))\
                   + (-self.f2.shape[0]/2)*np.log(maxf)\
                   + self.f1.T.dot(self.mu)/(2*self.sigma2)
        self.l[i] = self.pi/(self.pi+(1-self.pi)*np.exp(trythis))

        if np.random.rand() < self.l[i]:
          self.beta[i] = [0] * self.group_size[i]
          self.Z[i] = 0
        else:
          self.beta[i] = np.random.multivariate_normal(self.mu.T, 
                                                       self.f2_inverse * self.sigma2)
          self.Z[i] = 1


    def _update_tau2(self, i):
        if self.Z[i] == 0:
          self.tau2[i] = np.random.gamma(shape=(self.group_size[i] + 1)/2,
                                         scale=2/self.lambda2)
        else:
          mean = np.sqrt(self.lambda2*self.sigma2/sum([self.beta[i][j]**2 for j in range(len(self.beta[i]))]))
          scale = self.lambda2
          self.tau2[i] = 1/stats.invgauss.rvs(mu=mean, scale=scale, random_state=42)


    def _update_sigma2(self):
        s = 0
        for i in range(self.ngroup):
            s += sum([self.beta[i][j]**2 for j in range(len(self.beta[i]))]) / self.tau2[i]

        self.sigma2 = stats.invgamma.rvs((self.n-1)/2 + sum(self.Z * self.group_size)/2 + self.alpha,
                                         scale=(self.YtY - 2*self.beta_vec.T.dot(self.XtY)\
                                                + self.beta_vec.dot(self.XtX).dot(self.beta_vec)\
                                                + s)/2 + self.gamma,
                                         random_state=42)


    def _update_pi(self):
        self.pi = np.random.beta(self.a + self.ngroup - sum(self.Z),
                                 self.b + sum(self.Z))
        
    def monte_carlo_EM_lambda2(self):
        self._initialize_EM_lambda()

        for update in range(self.num_update):
            coef = np.zeros((self.p, self.niter_update))
            tau2_each_update = np.zeros((self.ngroup, self.niter_update))

            for iter in range(self.niter_update):
                if self.verbose:
                    print('EM for lambda2:', iter)
                
                # Update beta's
                for i in range(self.ngroup):
                    self._update_beta(i)
                
                # Update tau2
                for i in range(self.ngroup):
                    self._update_tau2(i)
                tau2_each_update[:, iter] = self.tau2

                # Create beta_vec
                self.beta_vec = np.concatenate([self.beta[i] for i in range(self.ngroup)])
                coef[:, iter] = self.beta_vec

                # Update sigma2
                self._update_sigma2()

                # Update pi
                self._update_pi()
            
            tau2_mean = np.mean(tau2_each_update, axis=1)

            self.lambda2 = np.sqrt((self.p + self.ngroup) / np.sum(tau2_mean))

            self.lambda2_path[update] = self.lambda2
        
        pos_mean = np.mean(coef, axis=1)
        pos_median = np.median(coef, axis=1)
        return self.lambda2

    def gibbs_sampler(self):
        coef = np.zeros((self.p, self.niter - self.burnin))
        coef_tau = np.zeros((self.ngroup, self.niter))
        
        self._get_matrix_computation()
        self.lambda2 = self.monte_carlo_EM_lambda2()
        self._initialize_gibbs_sampler()
        

        for iter in range(self.niter):
            if self.verbose == True:
                print(iter)

            # Update beta
            for i in range(self.ngroup):
                self._update_beta(i)
            
            # Update tau2'set
            for i in range(self.ngroup):
                self._update_tau2(i)
                coef_tau[i, iter] = self.tau2[i]
            
            self.beta_vec = np.concatenate([self.beta[i] for i in range(self.ngroup)])
            if iter > self.burnin:
                coef[:, iter - self.burnin] = self.beta_vec

            # Update sigma2
            self._update_sigma2()

            # Update pi
            self._update_pi()
        
        pos_mean = np.mean(coef, axis=1)
        pos_median = np.median(coef, axis=1)

        return pos_mean, pos_median

In [9]:
bglss1 = BGLSS(Y=Y1, X=X1, group_size=group_size1)
pos_mean1, pos_median1 = bglss1.gibbs_sampler()
print('Median: ', pos_median1)
print('\nReal beta: ', beta1)

Median:  [ 0.         -1.56441414  1.64451703  2.61624189  1.6013413   0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]

Real beta:  [[ 0.3  -1.    0.    0.5   0.01  0.    0.    0.    0.    0.    0.8   0.8
   0.8   0.8   0.8   0.    0.    0.    0.    0.  ]]


In [10]:
bglss2 = BGLSS(Y=Y2, X=X2, group_size=group_size2)
pos_mean2, pos_median2 = bglss2.gibbs_sampler()
print('Median: ', pos_median2)
print('\nReal beta: ', beta2)

Median:  [1.05543226 2.51687248 2.76750215 3.43305005 4.8422911  0.
 0.         0.         0.         0.         0.         0.05187517
 0.53356896 0.44501294 0.70791417 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]

Real beta:  [[1.  2.  3.  4.  5.  0.  0.  0.  0.  0.  0.1 0.2 0.3 0.4 0.5 0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. 

In [11]:
bglss3 = BGLSS(Y=Y3, X=X3, group_size=group_size3)
pos_mean3, pos_median3 = bglss3.gibbs_sampler()
print('Median: ', pos_median3)
print('\nReal beta: ', beta3)

Median:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         2.05974656 1.89236091
 1.92605676 1.90545534 2.35828573 2.13291814 1.76622306 1.76696159
 2.32900077 2.055836   0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 2.14380158 2.4185438  1.98551604 1.82904827 1.99504175 2.28752207
 1.66684332 2.12737156 1.61027325 1.87892349]

Real beta:  [[0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2
  2 2 2 2]]


In [12]:
bglss4 = BGLSS(Y=Y4, X=X4, group_size=group_size4)
pos_mean4, pos_median4 = bglss4.gibbs_sampler()
print('Median: ', pos_median4)
print('\nReal beta: ', beta4)

Median:  [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.90639799  1.81040819
  2.51087109  1.47206473  2.37356009  0.04195677  0.18272525  0.16962479
 -0.1251452  -0.11823972  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  1.83244457  2.39505209  2.55320701  2.18501761  1.61390542  0.03611094
 -0.08899055 -0.08778826  0.02165591 -0.46009407]

Real beta:  [[0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 0
  0 0 0 0]]


#### Compare Performances

In [13]:
class ComparePredictionPerformance():
    def __init__(self, n_simu):
        self.n_simu = n_simu
    
    def _simulate_data_ex1(self):
        self.beta1 = np.array([[0.3, -1, 0, 0.5, 0.01] + [0]*5 + [0.8]*5 + [0]*5])
        self.example = SimulateDataSet(n_obs=100, n_cov=20, n_groups=4, n_train=60, 
                           beta=self.beta1, corr=0.5, sigma=3, example=1)
        self.group_size = [5] * 4
        self.group_index = np.array([1] * 5 + [2] * 5 + [3] * 5 + [4] * 5)
    
    def _simulate_data_ex2(self):
        self.beta2 = np.array([[1, 2, 3, 4, 5] + [0]*5 + [0.1, 0.2, 0.3, 0.4, 0.5] + 13*([0]*5)])
        self.example = SimulateDataSet(n_obs=60, n_cov=80, n_groups=16, n_train=40, 
                           beta=self.beta2, corr=None, sigma=2, example=2)
        self.group_size = [5] * 16
        self.group_index = []
        for i in range(1, 17):
            self.group_index.extend([i]*5)
        self.group_index = np.array(self.group_index)
    
    def _simulate_data_ex3(self):
        self.beta3 = np.array([[0]*10 + [2]*10 + [0]*10 + [2]*10])
        self.example = SimulateDataSet(n_obs=100, n_cov=40, n_groups=4, n_train=60, 
                           beta=self.beta3, corr=None, sigma=2, example=2)
        self.group_size = [10] * 4
        self.group_index = np.array([1]*10 + [2]*10 + [3]*10 + [4]*10)

    def _simulate_data_ex4(self):
        self.beta4 = np.array([[0]*10 + [2]*5 + [0]*5 + [0]*10 + [2]*5 + [0]*5])
        self.example = SimulateDataSet(n_obs=100, n_cov=40, n_groups=4, n_train=60, 
                           beta=self.beta4, corr=None, sigma=2, example=2)
        self.group_size = [10] * 4
        self.group_index = np.array([1]*10 + [2]*10 + [3]*10 + [4]*10)
    
    def _compute_group_lasso(self, penalization):
        # Define a cross validation object to compute best lambda
        cv_class = asgl.CV(model='lm', penalization=penalization,
                           lambda1=(10.0 ** np.arange(-3, 1.51, 0.2)),
                           nfolds=5, error_type='MSE', parallel=True, 
                           random_state=42, alpha = np.arange(0, 1, 0.05))
        # Compute error using k-fold cross validation
        shape_y = len(self.y_train)
        error = cv_class.cross_validation(x=self.X_train, 
                                          y=self.y_train.reshape(shape_y,), 
                                          group_index=self.group_index)
        # Obtain the mean error across different folds
        error = np.mean(error, axis=1)
        # Select the minimum error
        minimum_error_idx = np.argmin(error)
        # Select the parameters associated to mininum error values
        optimal_parameters = cv_class.retrieve_parameters_value(minimum_error_idx)
        optimal_lambda = optimal_parameters.get('lambda1')

        # Define asgl class using optimal values
        self.gl_model = asgl.ASGL(model='lm', 
                                  penalization=penalization, 
                                  lambda1=optimal_lambda)

        return self.gl_model
        

    def pred_performance(self):
        self.final_scores = pd.DataFrame()
        self.feature_selection_final = pd.DataFrame()
        for num_ex in range(1, 5):
            print('--- Example ' + str(num_ex) + ' ---')
            self.scores = {'BGL-SS with mean': [], 'BGL-SS with median': [],
                           'Group Lasso': [], 'Sparse Group Lasso': [],
                           'Linear Regression': []}
            self.feature_selection = {'BGL-SS with mean': [], 'BGL-SS with median': [],
                                      'Group Lasso': [], 'Sparse Group Lasso': [],
                                      'Linear Regression': []}

            for i in range(self.n_simu):
                if i%10 == 0:
                    print('Simulation ' + str(i) + '...')
                if num_ex == 1:
                    self._simulate_data_ex1()
                elif num_ex == 2:
                    self._simulate_data_ex2()
                elif num_ex == 3:
                    self._simulate_data_ex3()
                else:
                    self._simulate_data_ex4()

                self.X = self.example.create_X()
                self.Y = self.example.simulate_Y()
                self.X_train, self.X_test, self.y_train, self.y_test = self.example.create_train_test_set()

                # Train bglss
                bglss = BGLSS(Y=self.y_train, X=self.X_train, group_size=self.group_size)
                self.beta_mean, self.beta_median = bglss.gibbs_sampler()
                
                # Compute bglss score with median
                y_pred = self.X_test.dot(self.beta_median)
                score = mean_squared_error(self.y_test, y_pred)
                self.scores['BGL-SS with median'].append(score)
                self.feature_selection['BGL-SS with median'].append(sum(self.beta_median == 0))

                # Compute bglss score with mean
                y_pred = self.X_test.dot(self.beta_mean)
                score = mean_squared_error(self.y_test, y_pred)
                self.scores['BGL-SS with mean'].append(score)
                self.feature_selection['BGL-SS with mean'].append(sum(self.beta_mean == 0))

                # Train Linear Regression
                lin_reg = LinearRegression()
                lin_reg.fit(self.X_train, self.y_train)
                y_pred = lin_reg.predict(self.X_test)
                score = mean_squared_error(self.y_test, y_pred)
                self.scores['Linear Regression'].append(score)
                self.feature_selection['Linear Regression'].append(sum(lin_reg.coef_[0] == 0))

                # Train Group Lasso
                gl_model = self._compute_group_lasso(penalization='gl')
                y_shape = len(self.y_train)
                gl_model.fit(x=self.X_train, y=self.y_train.reshape(y_shape,), group_index=self.group_index)
                y_pred = gl_model.predict(x_new=self.X_test)
                score = mean_squared_error(self.y_test, y_pred[0])
                self.scores['Group Lasso'].append(score)
                self.feature_selection['Group Lasso'].append(sum(gl_model.coef_[0] == 0))

                # Train Sparse Group Lasso
                sgl_model = self._compute_group_lasso(penalization='sgl')
                y_shape = len(self.y_train)
                sgl_model.fit(x=self.X_train, y=self.y_train.reshape(y_shape,), group_index=self.group_index)
                y_pred = sgl_model.predict(x_new=self.X_test)
                score = mean_squared_error(self.y_test, y_pred[0])
                self.scores['Sparse Group Lasso'].append(score)
                self.feature_selection['Sparse Group Lasso'].append(sum(sgl_model.coef_[0] == 0))
            
            self.scores = pd.DataFrame(pd.DataFrame(self.scores).mean(axis=0))
            self.scores.columns = ['Example ' + str(num_ex)]
            self.final_scores = pd.concat([self.final_scores, self.scores], axis=1)

            self.feature_selection = pd.DataFrame(pd.DataFrame(self.feature_selection).mean(axis=0))
            self.feature_selection.columns = ['Example ' + str(num_ex)]
            self.feature_selection_final = pd.concat([self.feature_selection_final, 
                                                      self.feature_selection], axis=1)
        
        return self.final_scores, self.feature_selection_final

            


In [14]:
prediction_performance = ComparePredictionPerformance(n_simu=50)
scores, feature_selection = prediction_performance.pred_performance()

--- Example 1 ---
Simulation 0...
Simulation 10...
Simulation 20...
Simulation 30...
Simulation 40...
--- Example 2 ---
Simulation 0...
Simulation 10...
Simulation 20...
Simulation 30...
Simulation 40...
--- Example 3 ---
Simulation 0...
Simulation 10...
Simulation 20...
Simulation 30...
Simulation 40...
--- Example 4 ---
Simulation 0...
Simulation 10...
Simulation 20...
Simulation 30...
Simulation 40...


In [17]:
scores

,Example 1,Example 2,Example 3,Example 4
BGL-SS with mean,9.267859,5.343971,5.815558,5.800728
BGL-SS with median,12.794051,5.364938,5.815895,5.795001
Group Lasso,9.253415,5.825239,6.153693,7.308652
Sparse Group Lasso,9.307444,6.079438,6.603958,6.700764
Linear Regression,13.867332,25.732770,14.077800,13.834890


In [18]:
feature_selection

,Example 1,Example 2,Example 3,Example 4
BGL-SS with mean,0.00,51.60,20.0,20.00
BGL-SS with median,15.74,70.54,20.0,20.00
Group Lasso,10.84,50.40,10.0,2.40
Sparse Group Lasso,12.68,54.04,12.2,12.78
Linear Regression,0.00,0.00,0.0,0.00
